In [1]:
import pickle
import gzip
import time

import numpy as np
import scipy as sp
import scipy.io as sio

import pymc3 as pm
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt
import pandas as pd
import random

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:

def get_data(sub_id):
    subdata=mydata[mydata.Subject==sub_id]
    #trialindex=list(subdata['trialindex'])
    phase=list(subdata['Block'])
    conds=list(subdata['Type'])
    rt=list(subdata['RT'])
    print('The trial length of sub%02d is %d' %(sub_id,len(phase)))
    return rt,phase,conds

In [3]:

def do_sampling_random_switchpoint(sub_id,rt,phase,conds,fakesp):
    # use empirical mean (ignoring condition or time point) as center of prior
    mu_obs = np.mean(rt)
    sd_obs = np.std(rt)
    
    model = pm.Model()
    with model:
       #mu_new is the mean RT of random stimulus
        mu_new = pm.Normal('mu_new', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        mu_before_old_benefit=pm.Normal('mu_before_old_benefit', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        mu_before = pm.math.switch(conds, mu_new-mu_before_old_benefit, mu_new)
        # RTs after switchpoint come from normal distribution where mean depends on
        # condition
        mu_after_old_benefit=pm.Normal('mu_after_old_benefit', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        #theano.tensor，switch(cond, ift, iff)，if cond then ift else iff
        mu_after = pm.math.switch(conds, mu_new-mu_after_old_benefit, mu_new)
        a=np.arange(fakesp,fakesp+1)
        switchpoint=a.repeat(len(phase)) 
        print('The fake switchpoint of sub%02d is %d' %(sub_id,fakesp))
        #if trial in or after the switchpoint session，mu=mu_after，else mu=mu_before
        mu = pm.math.switch(phase > switchpoint-1, mu_after, mu_before)
        
        sigma = pm.HalfNormal('sigma', sd=sd_obs*2, testval=sd_obs*2)
           #model construction
        rt_modelled = pm.Normal('rt_modelled', mu=mu, sd=sigma, observed=rt)
        
        step = pm.Metropolis()
        
        trace = pm.sample(40000, step=step, start=model.test_point, chains=4,cores=4)#MCMC
    
    return trace[20000::5], model#delete the first 20000 samples(burn-in)，take every fifth of the remaining samples(thining)

In [4]:
def do_sampling_given_switchpoint(sub_id,rt,phase, conds,sp):
    # use empirical mean (ignoring condition or time point) as center of prior
    mu_obs = np.mean(rt)
    sd_obs = np.std(rt)
    
    model = pm.Model()
    with model:
        # RTs before switchpoint all come from same distribution
        mu_new = pm.Normal('mu_new', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        mu_before_old_benefit=pm.Normal('mu_before_old_benefit', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        mu_before = pm.math.switch(conds, mu_new-mu_before_old_benefit, mu_new)
        # RTs after switchpoint come from normal distribution where mean depends on
        # condition
        mu_after_old_benefit=pm.Normal('mu_after_old_benefit', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        #theano.tensor，switch(cond, ift, iff)，if cond then ift else iff
        mu_after = pm.math.switch(conds, mu_new-mu_after_old_benefit, mu_new)
        a=np.arange(sp,sp+1)
        switchpoint=a.repeat(len(phase)) 
        print('The true switchpoint of sub%02d is %d' %(sub_id,sp))
        mu = pm.math.switch(phase >switchpoint-1, mu_after, mu_before)
        
        sigma = pm.HalfNormal('sigma', sd=sd_obs*2, testval=sd_obs*2)
        rt_modelled = pm.Normal('rt_modelled', mu=mu, sd=sigma, observed=rt)
        
        step = pm.Metropolis()
        
        trace = pm.sample(40000, step=step, start=model.test_point, chains=4,cores=4)

    return trace[20000::5], model

In [5]:
def do_sampling_noswitchpoint(sub_id,rt, phase, conds):
    # use empirical mean (ignoring condition or time point) as center of prior
    mu_obs = np.mean(rt)
    sd_obs = np.std(rt)
    
    model = pm.Model()
    with model:
        mu_new =  pm.Normal('mu_new', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        mu_old_benefit =  pm.Normal('mu_old_benefit', mu=mu_obs, sd=sd_obs*2, testval=mu_obs)
        sigma = pm.HalfNormal('sigma', sd=sd_obs*2, testval=sd_obs*2)
        
        mu = pm.math.switch(conds, mu_new-mu_old_benefit, mu_new)
        
        rt_modelled = pm.Normal('rt_modelled', mu=mu, sd=sigma, observed=rt)
        
        step = pm.Metropolis()
        
        trace = pm.sample(40000, step=step, start=model.test_point, chains=4,cores=4)
    return trace[20000::5], model

In [6]:
def model_construct(sub_id,model_type,sp):
        rt,phase,conds=get_data(sub_id)
        logrt=np.log10(rt)
        plt.scatter(list(range(1,len(logrt)+1)),logrt)
        plt.savefig(filepath+'/scatter_sub{:02d}.png'.format(sub_id))
        print("Now is fitting %s model for sub%02d......"%(model_type,sub_id))
        if model_type=='nosp':
            trace,model=do_sampling_noswitchpoint(sub_id,logrt,phase,conds)
        elif model_type=='randomsp':
            trace,model=do_sampling_random_switchpoint(sub_id,logrt,phase,conds,sp)
        elif model_type=='givensp':
            trace,model=do_sampling_given_switchpoint(sub_id,logrt,phase,conds,sp)
        with model:
            pm.traceplot(trace)
            plt.savefig(filepath+'/{}_trace_sub{:02d}.png'.format(model_type, sub_id))
            plt.close('all')
            
            pm.plot_posterior(trace)
            plt.savefig(filepath+'/{}_posterior_sub{:02d}.png'.format(model_type, sub_id))
            plt.close('all')
              
            #export data
            with gzip.open(filepath + '/tracedata/{}_trace_sub{:02d}.pkl.gz'.format(model_type, sub_id), 'wb') as f:
                pickle.dump((trace, model), f)
            waic=pm.waic(trace,scale='deviance')
        print("The WAIC of %s model is %f"%(model_type,waic.waic))
        print("--------------------------------------------------------")
        return trace,model


In [7]:
def run(sub_id,sp,fakesp):
    tracenp,modelnp=model_construct(sub_id,'nosp',sp)
    tracegp,modelgp=model_construct(sub_id,'givensp',sp)
    tracerp,modelrp=model_construct(sub_id,'randomsp',fakesp)
    with pd.ExcelWriter(filepath+'/summary_sub'+str(sub_id)+'.xlsx') as writer: 
        with modelnp:
            pm.summary(tracenp).to_excel(writer, sheet_name='noswitchpoint')
           
        with modelgp:
            pm.summary(tracegp).to_excel(writer, sheet_name='givenswtichpoint')

        with modelrp:
            pm.summary(tracerp).to_excel(writer, sheet_name='randomswitchpoint')
    df_comp_WAIC = pm.compare({'randomswitchpoint': tracerp,'noswitchpoint': tracenp,'givenswitchpoint':tracegp},ic='waic',scale='deviance')
    df_comp_WAIC.to_csv(filepath+'/cmp_waic_sub'+str(sub_id)+'.csv')
    

In [10]:

tpdata=pd.read_csv('E:/transition-upload/Python/Bayesian model/experiment1/exp1_tp.csv')
tpdata.subject=(tpdata['subject']).astype(int)
tpdata.transition_Phase=(tpdata['true_transition']).astype(int)
tpdict =tpdata.set_index('subject')['true_transition'].to_dict()
print(tpdict)
ftpdict =tpdata.set_index('subject')['fake_transition'].to_dict()
print(ftpdict)

{44: 6, 48: 5, 50: 4, 56: 5, 57: 4, 58: 4, 61: 5, 64: 4, 65: 5, 66: 4, 68: 4, 71: 6, 75: 7}
{44: 3, 48: 3, 50: 6, 56: 7, 57: 7, 58: 3, 61: 4, 64: 3, 65: 6, 66: 3, 68: 5, 71: 4, 75: 5}


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#read RT data
mydata=pd.read_csv('E:/transition-upload/Python/Bayesian model/experiment1/exp1_expdata.csv')
#delete NULL data（trials that ACC=0）
mydata.dropna(axis=0,how='any',inplace=True)
print(mydata)


       Subject  Block  Session  ACC  Type     RT  trailindex
0            1      1        1    1  True  503.0           1
1            1      1        1    1  True  526.0           2
2            1      1        1    1  True  736.0           3
3            1      1        1    1  True  576.0           4
4            1      1        1    1  True  676.0           5
...        ...    ...      ...  ...   ...    ...         ...
22195       78      9        9    1  True  343.0         544
22196       78      9        9    1  True  377.0         545
22197       78      9        9    1  True  423.0         546
22198       78      9        9    1  True  382.0         547
22199       78      9        9    1  True  450.0         548

[21546 rows x 7 columns]


In [12]:
#output filepath
filepath='E:/transition-upload/Python/Bayesian model/experiment1/'

In [17]:
#run
subid=1
if subid in tpdict.keys():
    run(subid,sp=tpdict[subid],fakesp=ftpdict[subid])
else:
    print("The subject is not in the list")

The subject is not in the list
